# Método para solução de sistemas lineares do tipo "Kd=F"

## Solução de sistemas invertendo a Matriz K

### Feito por: Leonardo Minski

In [721]:
import numpy as np
from IPython.display import display, Math, HTML


In [722]:
E = 3000 #kN/cmˆ2
bw = 15 # cm
h = 40 # cm
I = bw*h**3/12# cmˆ4
L = 100 # cm
P = 50 #kN

Graus_de_Liberdade_Restringidos = [0,4] # Python começa no zero


Ke = np.array([[12*E*I/L**3, 6*E*I/L**2, -12*E*I/L**3, 6*E*I/L**2],
      [6*E*I/L**2,  4*E*I/L,    -6*E*I/L**2,  2*E*I/L],
      [-12*E*I/L**3, -6*E*I/L**2, 12*E*I/L**3, -6*E*I/L**2],
      [6*E*I/L**2, 2*E*I/L, -6*E*I/L**2, 4*E*I/L]])

print(Ke)

[[ 2.88e+03  1.44e+05 -2.88e+03  1.44e+05]
 [ 1.44e+05  9.60e+06 -1.44e+05  4.80e+06]
 [-2.88e+03 -1.44e+05  2.88e+03 -1.44e+05]
 [ 1.44e+05  4.80e+06 -1.44e+05  9.60e+06]]


In [723]:
#Número de nós é igual a 3 e o número de graus de liberdade é igual a 6
Num_Barras = 2
Numero_Nos = Num_Barras + 1 
Graus_Liberdade = Numero_Nos * 2 # Graus de liberdade, no caso é deslocamento, rotação, deslocamento, rotação e assim por diante...

# Matriz_geral = [] # Matriz principal onde vão ser adicionados os valores
# for i in range(Graus_Liberdade): # Matriz quadrada
#    Matriz_geral.append(np.zeros(Graus_Liberdade))

Matriz_geral = np.zeros((Graus_Liberdade, Graus_Liberdade))

Matriz_geral # Matriz de rigidez geral


array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [724]:
# Lista para armazenar as matrizes de rigidez locais
Matriz_Rigidez_Local = []

# Loop para adicionar as matrizes de rigidez local à matriz de rigidez global
for i in range(Num_Barras):
    # Cria uma cópia da matriz de rigidez local
    Matriz_Rigidez_Local_i = np.copy(Matriz_geral)
    
    # Atualiza a submatriz correspondente da matriz de rigidez local na matriz de rigidez global
    Matriz_Rigidez_Local_i[i *  2 : 4 + i * 2, i * 2 : 4 + i * 2] += Ke
    
    # Adiciona a matriz de rigidez local à lista
    Matriz_Rigidez_Local.append(Matriz_Rigidez_Local_i)

for lista in Matriz_Rigidez_Local:
    Matriz_geral = Matriz_geral + lista

Matriz_geral_completa = Matriz_geral

In [731]:
Matriz_geral_completa

array([[ 2.88e+03,  1.44e+05, -2.88e+03,  1.44e+05,  0.00e+00,  0.00e+00],
       [ 1.44e+05,  9.60e+06, -1.44e+05,  4.80e+06,  0.00e+00,  0.00e+00],
       [-2.88e+03, -1.44e+05,  5.76e+03,  0.00e+00, -2.88e+03,  1.44e+05],
       [ 1.44e+05,  4.80e+06,  0.00e+00,  1.92e+07, -1.44e+05,  4.80e+06],
       [ 0.00e+00,  0.00e+00, -2.88e+03, -1.44e+05,  2.88e+03, -1.44e+05],
       [ 0.00e+00,  0.00e+00,  1.44e+05,  4.80e+06, -1.44e+05,  9.60e+06]])

In [725]:
Graus_de_Liberdade_Restringidos.sort(reverse=True) # invertendo para não remover termos errados

Matriz_geral = np.array(Matriz_geral)
Matriz_geral = np.delete(Matriz_geral, Graus_de_Liberdade_Restringidos, axis=0) # Removendo linha
Matriz_geral = np.delete(Matriz_geral, Graus_de_Liberdade_Restringidos, axis=1) # Removendo coluna

Matriz_geral # Removido as linhas e colunas não necessárias

array([[ 9.60e+06, -1.44e+05,  4.80e+06,  0.00e+00],
       [-1.44e+05,  5.76e+03,  0.00e+00,  1.44e+05],
       [ 4.80e+06,  0.00e+00,  1.92e+07,  4.80e+06],
       [ 0.00e+00,  1.44e+05,  4.80e+06,  9.60e+06]])

In [726]:
Matriz_geral_invertida = np.linalg.inv(Matriz_geral) # K elevado a -1, inversão de matriz.

Matriz_geral_invertida

array([[ 2.77777778e-07,  1.04166667e-05, -3.47222222e-08,
        -1.38888889e-07],
       [ 1.04166667e-05,  6.94444444e-04,  0.00000000e+00,
        -1.04166667e-05],
       [-3.47222222e-08,  0.00000000e+00,  6.94444444e-08,
        -3.47222222e-08],
       [-1.38888889e-07, -1.04166667e-05, -3.47222222e-08,
         2.77777778e-07]])

In [727]:
F = [0, 0, -P, 0, 0, 0] # Remover via GDL da lista

[F.pop(index) for index in sorted(Graus_de_Liberdade_Restringidos, reverse=True)]


# Multiplicar cada linha da matriz pelo vetor
Deslocamentos = [np.dot(linha, F) for linha in Matriz_geral_invertida]

Deslocamentos

[-0.0005208333333333332, -0.03472222222222222, 0.0, 0.0005208333333333332]

In [728]:
desloc_aux = np.zeros((Graus_Liberdade))

idx = 0
for index in range(Graus_Liberdade):
    if index not in Graus_de_Liberdade_Restringidos:
        desloc_aux[index] = Deslocamentos[idx]
        idx += 1

desloc_aux # Adicionados os graus de liberdade igual a zero para os outros cálculos

array([ 0.        , -0.00052083, -0.03472222,  0.        ,  0.        ,
        0.00052083])

In [729]:
ReacoesApoio = np.array(Matriz_geral_completa) @ np.array(desloc_aux)
ReacoesApoio

array([ 2.50000000e+01,  0.00000000e+00, -5.00000000e+01, -1.00364161e-13,
        2.50000000e+01, -2.00728323e-13])

In [730]:
Cortante_i = []
Momento_i = []

Cortante_j = []
Momento_j = []

for i in range(Matriz_Rigidez_Local.__len__()):
    inicio = i*2
    fim = i * 2 + 4
    vetor_resultado = np.array(Matriz_Rigidez_Local[i][inicio:fim]) @ np.array(desloc_aux)
    print(vetor_resultado)
    #Cortante_i.append(vetor_resultado[0])
    #Momento_i.append(vetor_resultado[1])
    #Cortante_j.append(vetor_resultado[2])
    #Momento_j.append(vetor_resultado[3])



[  25.    0.  -25. 2500.]
[-2.50000000e+01 -2.50000000e+03  2.50000000e+01 -2.00728323e-13]
